In [1]:
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

In [2]:
df=pd.read_excel('Course Project - Data for Classification - Electric Vehicles.xls')
#df.head()

In [3]:
#print(df.dtypes)

In [4]:
#print(df['Q16'].unique()) 


In [5]:
#df['Q16'].value_counts() 

In [6]:
#df.head()

In [7]:
# Replace non-numeric values with NaN
df.replace('?', np.nan, inplace=True)
df.head()

,Q11_1,Q11_2,Q11_3,Q11_4,Q11_5,Q11_6,Q11_7,Q11_8,Q11_9,Q11_10,...,Q18_17,Q18_18,Q18_19,Q18_20,Q18_21,Q18_22,Q18_23,Q20,Q21,Q16
0,0,0,0,1,1,0,0,1,0,0,...,5.0,5.0,5.0,5.0,5.0,5.0,5.0,3,3,A
1,1,0,0,1,0,0,1,0,0,1,...,4.0,2.0,4.0,2.0,2.0,2.0,2.0,3,3,D
2,1,1,0,1,0,0,1,1,0,1,...,4.0,4.0,NaN,NaN,4.0,2.0,4.0,4,1,A
3,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,3,C
4,1,1,0,1,0,0,1,1,0,1,...,NaN,NaN,1.0,1.0,NaN,NaN,NaN,3,3,D


In [8]:
from sklearn.impute import KNNImputer

# Assuming 'df' is your DataFrame and 'exclude_column_name' is the column to exclude
exclude_column_name = 'Q16'
column_to_exclude = df[exclude_column_name]

# Create a list of k values
k_values = [1, 3, 5, 7, 9,11]

# Loop through each k value
for k in k_values:
    # Drop the column you want to exclude
    columns_to_impute = df.drop(columns=[exclude_column_name])
    
    # Apply KNN imputation for the current k value
    imputer = KNNImputer(n_neighbors=k)
    imputed_data = imputer.fit_transform(columns_to_impute)
    
    
    imputed_df = pd.DataFrame(imputed_data, columns=columns_to_impute.columns)
    imputed_df[exclude_column_name] = column_to_exclude


In [31]:
imputed_df.head()
scaled_columns = imputed_df.iloc[:, :-1].apply(lambda x: (x - x.min()) / (x.max() - x.min()) if pd.api.types.is_numeric_dtype(x) else x, axis=0)

# Concatenate the original DataFrame with the scaled columns
imputed_df = pd.concat([scaled_columns, imputed_df['Q16']], axis=1)

# Convert values to binary (0 or 1) based on the condition
imputed_df.iloc[:, :-1] = imputed_df.iloc[:, :-1].apply(lambda x: x.map(lambda val: 1 if pd.notna(val) and float(val) > 0.70 else 0))

# Convert to integer to make them binary
imputed_df.iloc[:, :-1] = imputed_df.iloc[:, :-1].astype(int)


imputed_df.head()


,Q11_1,Q11_2,Q11_3,Q11_4,Q11_5,Q11_6,Q11_7,Q11_8,Q11_9,Q11_10,...,Q18_17,Q18_18,Q18_19,Q18_20,Q18_21,Q18_22,Q18_23,Q20,Q21,Q16
0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,A
1,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,D
2,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,...,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,A
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,C
4,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,D


In [32]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import PolynomialFeatures, RobustScaler
from scipy import stats

# Özellik ve hedef değişkeni seç
X = imputed_df.drop('Q16', axis=1)
y = imputed_df['Q16']

# Aykırı değerleri ele
z_scores = stats.zscore(X)
abs_z_scores = np.abs(z_scores)
filtered_entries = (abs_z_scores < 3).all(axis=1)
X = X[filtered_entries]
y = y[filtered_entries]

# Genişletilmiş özellik setini oluştur
poly = PolynomialFeatures(degree=2)
X_poly = poly.fit_transform(X)

# Özellikleri RobustScaler ile ölçeklendirme
scaler = RobustScaler()
X_scaled = scaler.fit_transform(X_poly)

# Veriyi train ve test setlerine ayırma
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


# Create RandomForestClassifier with the best parameters
rf_classifier = RandomForestClassifier(max_depth= 10, min_samples_leaf= 2, min_samples_split= 5, n_estimators= 200)

# Modeli eğitin
rf_classifier.fit(X_train, y_train)

# Test seti üzerinde tahminler yapın
y_pred = rf_classifier.predict(X_test)

# Doğruluk skorunu değerlendirin
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.4f}")


Model Accuracy: 0.6497


In [11]:
'''from sklearn.preprocessing import PolynomialFeatures
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

# Özellik ve hedef değişkeni seç
X = imputed_df.drop('Q16', axis=1)
y = imputed_df['Q16']

# Aykırı değerleri ele
z_scores = stats.zscore(X)
abs_z_scores = np.abs(z_scores)
filtered_entries = (abs_z_scores < 3).all(axis=1)
X = X[filtered_entries]
y = y[filtered_entries]

# Genişletilmiş özellik setini oluştur
poly = PolynomialFeatures(degree=2)
X_poly = poly.fit_transform(X)

# Özellikleri RobustScaler ile ölçeklendirme
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_poly)

# Veriyi train ve test setlerine ayırma
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Logistic Regression modeli
logreg = LogisticRegression(max_iter=1000)

# RFECV'yi kullanarak özellik seçimi yapın
rfecv = RFECV(estimator=logreg, step=1, cv=5, scoring='accuracy')  # cv=5, 5-fold cross-validation kullanıyoruz
X_train_rfecv = rfecv.fit_transform(X_train, y_train)

# Modeli eğitin
logreg.fit(X_train_rfecv, y_train)

# Test seti üzerinde tahminler yapın
X_test_rfecv = rfecv.transform(X_test)
y_pred = logreg.predict(X_test_rfecv)

# Doğruluk skorunu değerlendirin
accuracy = accuracy_score(y_test, y_pred)
print(f"Logistic Regression Model Accuracy with RFECV: {accuracy:.4f}")

# Modelin performansını 5 katlı çapraz doğrulama ile değerlendirin
cv_scores = cross_val_score(logreg, X_train_rfecv, y_train, cv=5, scoring='accuracy')

# Her bir çapraz doğrulama seti için doğruluk skorlarını yazdır
for i, score in enumerate(cv_scores, start=1):
    print(f"Çapraz Doğrulama Seti {i}: Doğruluk = {score}")

# Ortalama doğruluk skorunu hesapla
mean_accuracy = cv_scores.mean()

print("\n5 Katlı Çapraz Doğrulama İle Ortalama Doğruluk:", mean_accuracy)'''


'from sklearn.preprocessing import PolynomialFeatures\nfrom sklearn.feature_selection import RFECV\nfrom sklearn.linear_model import LogisticRegression\nfrom sklearn.model_selection import train_test_split, cross_val_score\nfrom sklearn.metrics import accuracy_score\nfrom sklearn.preprocessing import StandardScaler\n\n# Özellik ve hedef değişkeni seç\nX = imputed_df.drop(\'Q16\', axis=1)\ny = imputed_df[\'Q16\']\n\n# Aykırı değerleri ele\nz_scores = stats.zscore(X)\nabs_z_scores = np.abs(z_scores)\nfiltered_entries = (abs_z_scores < 3).all(axis=1)\nX = X[filtered_entries]\ny = y[filtered_entries]\n\n# Genişletilmiş özellik setini oluştur\npoly = PolynomialFeatures(degree=2)\nX_poly = poly.fit_transform(X)\n\n# Özellikleri RobustScaler ile ölçeklendirme\nscaler = StandardScaler()\nX_scaled = scaler.fit_transform(X_poly)\n\n# Veriyi train ve test setlerine ayırma\nX_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)\n\n# Logistic Regression mod

In [12]:
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Önceden belirlenmiş X ve y'yi kullanalım
# X: Özellik matrisi, y: Hedef değişken
# Örneğin, new_X ve new_y'yi kullanalım

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Logistic Regression modeli
logreg = LogisticRegression(max_iter=1000)

# RFECV'yi kullanarak özellik seçimi yapın
rfecv = RFECV(estimator=logreg, step=1, cv=5, scoring='accuracy')  # cv=5, 5-fold cross-validation kullanıyoruz
X_train_rfecv = rfecv.fit_transform(X_train, y_train)

# Modeli eğitin
logreg.fit(X_train_rfecv, y_train)

# Test seti üzerinde tahminler yapın
X_test_rfecv = rfecv.transform(X_test)
y_pred = logreg.predict(X_test_rfecv)

# Doğruluk skorunu değerlendirin
accuracy = accuracy_score(y_test, y_pred)
print(f"Logistic Regression Model Accuracy with RFECV: {accuracy:.4f}")

###############################################################################################

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

X_train_scaled = scaler.fit_transform(X_train_rfecv)
X_test_scaled = scaler.transform(X_test_rfecv)

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

logreg.fit(X_train_scaled, y_train)
y_pred_scaled = logreg.predict(X_test_scaled)

accuracy_scaled = accuracy_score(y_test, y_pred_scaled)
print(f"Logistic Regression Model Accuracy with RFECV and Scaling: {accuracy_scaled:.4f}")

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

# Random Forest
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train_scaled, y_train)
y_pred_rf = rf_model.predict(X_test_scaled)

accuracy_rf = accuracy_score(y_test, y_pred_rf)
print(f"Random Forest Model Accuracy: {accuracy_rf:.4f}")

# Gradient Boosting
gb_model = GradientBoostingClassifier(random_state=42)
gb_model.fit(X_train_scaled, y_train)
y_pred_gb = gb_model.predict(X_test_scaled)

accuracy_gb = accuracy_score(y_test, y_pred_gb)
print(f"Gradient Boosting Model Accuracy: {accuracy_gb:.4f}")



Logistic Regression Model Accuracy with RFECV: 0.6820
Logistic Regression Model Accuracy with RFECV and Scaling: 0.6820
Random Forest Model Accuracy: 0.6868
Gradient Boosting Model Accuracy: 0.6916


In [13]:
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Önceden belirlenmiş X ve y'yi kullanalım
# X: Özellik matrisi, y: Hedef değişken
# Örneğin, new_X ve new_y'yi kullanalım

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Logistic Regression modeli
logreg = LogisticRegression(max_iter=1000)

# RFECV'yi kullanarak özellik seçimi yapın
rfecv = RFECV(estimator=logreg, step=1, cv=5, scoring='accuracy')  # cv=5, 5-fold cross-validation kullanıyoruz
X_train_rfecv = rfecv.fit_transform(X_train, y_train)

# Modeli eğitin
logreg.fit(X_train_rfecv, y_train)

# Test seti üzerinde tahminler yapın
X_test_rfecv = rfecv.transform(X_test)
y_pred = logreg.predict(X_test_rfecv)

# Doğruluk skorunu değerlendirin
accuracy = accuracy_score(y_test, y_pred)
print(f"Logistic Regression Model Accuracy with RFECV: {accuracy:.4f}")

###############################################################################################


from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

X_train_scaled = scaler.fit_transform(X_train_rfecv)
X_test_scaled = scaler.transform(X_test_rfecv)

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

logreg.fit(X_train_scaled, y_train)
y_pred_scaled = logreg.predict(X_test_scaled)

accuracy_scaled = accuracy_score(y_test, y_pred_scaled)
print(f"Logistic Regression Model Accuracy with RFECV and Scaling: {accuracy_scaled:.4f}")

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train_scaled, y_train)
y_pred_rf = rf_model.predict(X_test_scaled)

accuracy_rf = accuracy_score(y_test, y_pred_rf)
print(f"Random Forest Model Accuracy with MinMax Scaling: {accuracy_rf:.4f}")

# Gradient Boosting
gb_model = GradientBoostingClassifier(random_state=42)
gb_model.fit(X_train_scaled, y_train)
y_pred_gb = gb_model.predict(X_test_scaled)

accuracy_gb = accuracy_score(y_test, y_pred_gb)
print(f"Gradient Boosting Model Accuracy with MinMax Scaling: {accuracy_gb:.4f}")


Logistic Regression Model Accuracy with RFECV: 0.6820
Logistic Regression Model Accuracy with RFECV and Scaling: 0.6820
Random Forest Model Accuracy with MinMax Scaling: 0.6868
Gradient Boosting Model Accuracy with MinMax Scaling: 0.6916


In [14]:
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import QuantileTransformer
import numpy as np
from scipy import stats
import numpy as np
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier



# Extract features and target variable
X = imputed_df.drop('Q16', axis=1)
y = imputed_df['Q16']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
##################################################################

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.utils.class_weight import compute_sample_weight
from collections import Counter
from sklearn.preprocessing import QuantileTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# Assuming you have already calculated class weights
class_weight_dict = {'A': 0.5246993127147767, 'B': 1.0078382838283828, 'C': 1.6918282548476453, 'D': 1.9575320512820513}
class_counts = Counter(y)
weighted_counts_df = pd.DataFrame({'Class': list(class_counts.keys()), 'Weighted Count': [class_counts[class_] * weight for class_, weight in class_weight_dict.items()]})
print("\nWeighted Class Counts:")
print(weighted_counts_df)
# Random Forest Classifier with class weights
rf_classifier = RandomForestClassifier(random_state=42, class_weight=class_weight_dict)
rf_classifier.fit(X_train, y_train)
y_pred_rf = rf_classifier.predict(X_test)
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print(f"Random Forest Accuracy with Class Weights: {accuracy_rf:.4f}")

# Support Vector Classifier (SVC) with class weights
svc_classifier = SVC(random_state=42, class_weight=class_weight_dict)
svc_classifier.fit(X_train, y_train)
y_pred_svc = svc_classifier.predict(X_test)
accuracy_svc = accuracy_score(y_test, y_pred_svc)
print(f"SVC Accuracy with Class Weights: {accuracy_svc:.4f}")

##################################################################


# Logistic Regression modeli
logreg = LogisticRegression(max_iter=1000, class_weight=class_weight_dict)

# RFECV'yi kullanarak özellik seçimi yapın
rfecv = RFECV(estimator=logreg, step=1, cv=5, scoring='accuracy')  # cv=5, 5-fold cross-validation kullanıyoruz
X_train_rfecv = rfecv.fit_transform(X_train, y_train)

quantile_transformer = QuantileTransformer(output_distribution='normal',random_state=0)
X_train_quantile = quantile_transformer.fit_transform(X_train_rfecv)
X_test_quantile = quantile_transformer.transform(rfecv.transform(X_test))


# Modeli eğitin
logreg.fit(X_train_quantile, y_train)

# Test seti üzerinde tahminler yapın
X_test_rfecv = rfecv.transform(X_test)
y_pred = logreg.predict(X_test_quantile)

# Doğruluk skorunu değerlendirin
accuracy = accuracy_score(y_test, y_pred)
print(f"Logistic Regression Model Accuracy with RFECV: {accuracy:.4f}")

###############################################################################################


from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()

X_train_scaled = scaler.fit_transform(X_train_quantile)
X_test_scaled = scaler.transform(X_test_quantile)


logreg.fit(X_train_scaled, y_train)
y_pred_scaled = logreg.predict(X_test_scaled)
accuracy_scaled = accuracy_score(y_test, y_pred_scaled)
print(f"Logistic Regression Model Accuracy with RFECV and Scaling: {accuracy_scaled:.4f}")

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train_scaled, y_train)
y_pred_rf = rf_model.predict(X_test_scaled)
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print(f"Random Forest Model Accuracy with MinMax Scaling: {accuracy_rf:.4f}")

# Gradient Boosting
gb_model = GradientBoostingClassifier(random_state=42)
gb_model.fit(X_train_scaled, y_train)
y_pred_gb = gb_model.predict(X_test_scaled)
accuracy_gb = accuracy_score(y_test, y_pred_gb)
print(f"Gradient Boosting Model Accuracy with MinMax Scaling: {accuracy_gb:.4f}")


Weighted Class Counts:
  Class  Weighted Count
0     A     1512.708119
1     D     1537.961221
2     C     1509.110803
3     B     1579.728365
Random Forest Accuracy with Class Weights: 0.6375
SVC Accuracy with Class Weights: 0.6170
Logistic Regression Model Accuracy with RFECV: 0.5835
Logistic Regression Model Accuracy with RFECV and Scaling: 0.5827
Random Forest Model Accuracy with MinMax Scaling: 0.6105
Gradient Boosting Model Accuracy with MinMax Scaling: 0.6620
